# DFX + DMA + self trigger

## import section

In [1]:
# import the library
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
import numpy as np
import os
import subprocess

PRJ_DIR    = '/home/xilinx/jupyter_notebooks/dfx4ml/'
PRJ_HW_DIR = '/home/xilinx/jupyter_notebooks/dfx4ml/hw/'

FULL_BS_NAME    = 'system.bin'
PAR_BS_NAME_0   = 'add.bin'
PAR_BS_NAME_1   = 'sub.bin'
PAR_BS_NAME_ML1 = 'ml1.bin'
PAR_BS_NAME_ML2 = 'ml2.bin'
PAR_BS_NAME_ML3 = 'ml3.bin'

AMT_QUERY       = 5
INPUT_SHAPE     = (AMT_QUERY, 6, 8 )
ML2IN_SHAPE     = (AMT_QUERY, 6, 16)
ML3IN_SHAPE     = (AMT_QUERY, 6, 8 )
DES_SHAPE       = (AMT_QUERY, 5)

## ICAP config

In [2]:
def subProcessPrint(prefix, subProcessRes):
    
    print(f"{prefix} STDOUT:", subProcessRes.stdout)
    print(f"{prefix} ERROR :", subProcessRes.stderr)
    print("--------------------------------")
    

def changePLconfigMode(mode, isRoot): ### mode should be "pcap", "icap"
    indicator = "1"
    
    if mode == "icap":
        indicator = "0"
    elif mode == "pcap":
        pass
    else:
        raise Exception("change PL config Mode has mode error")
    
    
    changeCmd_inputVal = f"0xFFCA3008 0xFFFFFFFF 0x{indicator}"
    triggerCmd_inputVal = "0xFFCA3008"
    config_file = "/sys/firmware/zynqmp/config_reg"
    password = "xilinx"  # default sudo password for PYNQ-ZU

    # Compose the command string
    cmd_change = f"sudo -S tee {config_file}"
    cmd_trigger = f"sudo -S tee {config_file}"
    cmd_read    = f"cat {config_file}"
    
    passwordCmd = (password + '\n') if not isRoot else ""

    # Run the change command
    result = subprocess.run(
        cmd_change,
        input=passwordCmd + changeCmd_inputVal + '\n',
        shell=True,
        capture_output=True,
        text=True
    )
    subProcessPrint("CHANGE CMD", result)

    # Run the trigger command
    result = subprocess.run(
        cmd_trigger,
        input=passwordCmd + triggerCmd_inputVal + '\n',
        shell=True,
        capture_output=True,
        text=True
    )

    subProcessPrint("TRIGGER CMD", result)

    
    result = subprocess.run(
        cmd_read,
        shell=True,
        capture_output=True,
        text=True
    )

    subProcessPrint("READ CMD", result)


changePLconfigMode("pcap", True)  # change the PL config mode to pcap

CHANGE CMD STDOUT: 0xFFCA3008 0xFFFFFFFF 0x1

CHANGE CMD ERROR : 
--------------------------------
TRIGGER CMD STDOUT: 0xFFCA3008

TRIGGER CMD ERROR : 
--------------------------------
READ CMD STDOUT: 0x1

READ CMD ERROR : 
--------------------------------


## Magic Sequence Driver

In [3]:
class MagicSeqdDriver (DefaultIP):

    def __init__(self, description):
        super().__init__(description=description)

        ### Bit Layout start bit
        self.BL_COL_ST  =  2 
        self.BL_ROW_ST  =  6 
        self.BL_BNK_ST  = 14 
        ### Bit Layout size
        self.BIT_COL_SZ = 4
        self.BIT_ROW_SZ = 8
        self.BIT_BNK_SZ = 2

        self.REG_CTRL     = (0,0,0)
        self.REG_ST       = (0,1,0)
        self.REG_MAINCNT  = (0,2,0)
        self.REG_ENDCNT   = (0,3,0)
        self.REG_DMA_ADDR = (0,4,0)
        self.REG_DFX_ADDR = (0,5,0)

        #### the row must be change to match the slot
        self.SLOT_SRC_ADDR = (1,0,0)
        self.SLOT_SRC_SIZE = (1,0,1)
        self.SLOT_DES_ADDR = (1,0,2)
        self.SLOT_DES_SIZE = (1,0,3)
        self.SLOT_STATUS   = (1,0,4)
        self.SLOT_PROF     = (1,0,5)


        self.LIM_AMT_SLOT = 4 ### limit amount slot
        

    bindto = ['user.org:user:MagicSeqTop:1.0']

    def genAddr(self, bankId, rowIdx, colIdx):
        return (bankId << self.BL_BNK_ST) | (rowIdx << self.BL_ROW_ST) | (colIdx << self.BL_COL_ST)
    
    def genAddrForSlot(self, slotT, slotIdx):
        return self.genAddr(slotT[0], slotIdx, slotT[2])

    ###############################################
    ####### getter ################################
    ###############################################

    def getStatus(self):
        return self.read(self.genAddr(*self.REG_ST))
    def getMainCnt(self):
        return self.read(self.genAddr(*self.REG_MAINCNT))
    def getEndCnt(self):
        return self.read(self.genAddr(*self.REG_ENDCNT))
    def getDmaAddr(self):
        return self.read(self.genAddr(*self.REG_DMA_ADDR))
    def getDfxAddr(self):
        return self.read(self.genAddr(*self.REG_DFX_ADDR))
    
    def getSlot(self, slotIdx):

        addr_srcAddr  = self.genAddrForSlot(self.SLOT_SRC_ADDR, slotIdx)             
        addr_srcSz    = self.genAddrForSlot(self.SLOT_SRC_SIZE, slotIdx)           
        addr_desAddr  = self.genAddrForSlot(self.SLOT_DES_ADDR, slotIdx)             
        addr_desSz    = self.genAddrForSlot(self.SLOT_DES_SIZE, slotIdx)           
        addr_status   = self.genAddrForSlot(self.SLOT_STATUS  , slotIdx)            
        addr_prof     = self.genAddrForSlot(self.SLOT_PROF    , slotIdx)

        data_srcAddr  = self.read(addr_srcAddr)
        data_srcSz    = self.read(addr_srcSz)
        data_desAddr  = self.read(addr_desAddr)
        data_desSz    = self.read(addr_desSz)
        data_status   = self.read(addr_status)
        data_prof     = self.read(addr_prof)

        return data_srcAddr, data_srcSz, data_desAddr, data_desSz, data_status, data_prof
        

    ###############################################
    ####### setter ################################
    ###############################################

    def setControl(self, value): #### status registesr will be neglect
        return self.write(self.genAddr(*self.REG_CTRL), value)
    # def setMainCnt(self, value):
    #     return self.write(self.genAddr(*self.REG_MAINCNT), value)
    def setEndCnt(self, value):
        return self.write(self.genAddr(*self.REG_ENDCNT), value)
    def setDmaAddr(self, value):
        return self.write(self.genAddr(*self.REG_DMA_ADDR), value)
    def setDfxAddr(self, value):
        return self.write(self.genAddr(*self.REG_DFX_ADDR), value)
    
    def setSlot(self, slotT, slotIdx, value):
        addr  = self.genAddrForSlot(slotT, slotIdx) 
        self.write(addr, value)

    def setWholeSlot(self, slotIdx, dataList):

        addr_srcAddr  = self.genAddrForSlot(self.SLOT_SRC_ADDR, slotIdx)             
        addr_srcSz    = self.genAddrForSlot(self.SLOT_SRC_SIZE, slotIdx)           
        addr_desAddr  = self.genAddrForSlot(self.SLOT_DES_ADDR, slotIdx)             
        addr_desSz    = self.genAddrForSlot(self.SLOT_DES_SIZE, slotIdx)           
        addr_status   = self.genAddrForSlot(self.SLOT_STATUS  , slotIdx)            
        addr_prof     = self.genAddrForSlot(self.SLOT_PROF    , slotIdx)

        self.write(addr_srcAddr, dataList[0])
        self.write(addr_srcSz  , dataList[1])
        self.write(addr_desAddr, dataList[2])
        self.write(addr_desSz  , dataList[3])
        self.write(addr_status , dataList[4])
        self.write(addr_prof   , dataList[5])

    ###############################################
    ####### command################################
    ###############################################

    def clearEngine(self):
        print("[cmd] clear the engine")
        self.setControl(0)
        print("[cmd] clear the engine successfully")


    def shutdownEngine(self):
        print("[cmd] shutdown the engine")
        self.setControl(1)
        print("[cmd] shutdown successfully")

    def startEngine(self):
        print("[cmd] start the engine")
        self.setControl(2)
        print("[cmd] start the successfully")

    ###############################################
    ####### debugger ##############################
    ###############################################

    def status2Str(self, statusIdx):
        mapper = ["SHUTDOWN","REPROG","W4SLAVERESET","W4SLAVEOP","INITIALIZING","TRIGGERING","WAIT4FIN","PAUSEONERROR"]

        if statusIdx not in range(0, len(mapper)):
            return "STATUS ERROR"
        return mapper[statusIdx]
        
    def printMainStatus(self):


        print("----- MAIN STATUS ------------------")
        status  = self.getStatus()
        print("--------> STATUS = ", self.status2Str(status))
        mainCnt = self.getMainCnt()
        print("--------> MAINCNT = ", mainCnt)
        endCnt  = self.getEndCnt()
        print("--------> ENDCNT  = ", endCnt)
        dmaAddr = self.getDmaAddr()
        print("--------> DMAADDR  = ", hex(dmaAddr))
        dfxAddr = self.getDfxAddr()
        print("--------> DFXADDR  = ", hex(dfxAddr))


    def printSlotData(self):

        print("----- SLOT DATA ------------------")

        if self.getStatus() != 0:
            print("---------- cannot print slot data due to the system is not in shutdown state")
            return

        for slotIdx in range (self.LIM_AMT_SLOT):
            s_addr, s_size, d_addr, d_size, status, prof = self.getSlot(slotIdx)

            print(f"------> slot {slotIdx} :")
            print(f"        srcAddr   : {hex(s_addr)},  srcSize   : {hex(s_size)}")
            print(f"        desAddr   : {hex(d_addr)},  desSize   : {hex(d_size)}")
            print(f"        status    : {hex(status)}")
            print(f"        profileCnt: {hex(prof)}")

    def printDebug(self):
        self.printMainStatus()
        self.printSlotData()
        print("-------------------------------")

## DFX controller Driver

In [4]:
class MyDfxCtrl(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        self.storage = None
        
        print(description)
        
        # BLS bit layout size
        self.BLS_DATA   = 2 # register contain 4 byte (2 bit addressing)
        self.BLS_REGID  = 4 # register Id
        self.BLS_BANKID = 2 # bank id
        #### TODO this may change 
        # GENERAL BANK
        self.BANK_GENREG      = 0     
        self.GENREG_STATUS    = 0
        self.GENREG_CTRL      = 0
        self.GENREG_SWTRIGGER = 1
        # TRIGGER RM MAPPING
        self.BANK_RMM         = 1
        self.BANK_RMM_LIMIT   = 2 #### it is not used now!
        # RM INFO
        self.BANK_RMINFO      = 2
        self.BANK_RMINFO_LIMIT= 2 #### it is not used now!
        # BITSTREAM INFO
        self.BANK_BSINFO      = 3
        self.BANK_BSINFO_LIMIT= 2 #### it is not used now!
        
        #print(description) #
        
    bindto = ['xilinx.com:ip:dfx_controller:1.0']
    
    ###### address generator
    def getAddress(self, bankId, regId): ### todo make it compatible for more than 1 slot
        return (bankId << (self.BLS_DATA + self.BLS_REGID)) + (regId << (self.BLS_DATA))
    #######################################
    ###### general command ################
    #######################################
    def shutdownEngine(self):
        print("shutdown dfx Controller")
        self.setCtrl(0)
    
    def restartNoStatus(self):
        print("restart the dfx Controller with no status")
        self.setCtrl(1)
        
    def restartWithStatus(self):
        print("restart the dfx Controller with status")
        
    def trigger(self, triggerId):
        print("trig the rmId ", triggerId)
        self.setCtrlTrigger(triggerId)

    #######################################
    ###### getter setter command ##########
    #######################################
        
    ###### general register bank0
    #
    # |statusRegister, controlRegister|
    # | trigger register              |
    #
    def getStatus(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_STATUS)
        print("[get status register] @", hex(regAddr))
        return self.read(regAddr)
    
    def getCtrl(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_CTRL)
        print("[get ctrl register] @", hex(regAddr))
        return self.read(regAddr)
    
    def setCtrl(self, command):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_CTRL)
        print("[set ctrl register] @", hex(regAddr), " with command ", hex(command))
        self.write(regAddr, command)
    
    def getCtrlTrigger(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_SWTRIGGER)
        print("[get Ctrl Trigger] @", hex(regAddr))
        return self.read(regAddr)
    
    def setCtrlTrigger(self, triggerId):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_SWTRIGGER)
        print("[set Ctrl Trigger] @", hex(regAddr))
        self.write(regAddr, triggerId)
        
    ####### Reconfig Module Map bank1
    # | RM0 |
    # | RM1 |
    # | RM2 |
    # .
    def getRMM(self, triggerId):
        regAddr = self.getAddress(self.BANK_RMM, triggerId)
        print("[get RM MAP] @", hex(regAddr))
        return self.read(regAddr)
    
    def setRMM(self, triggerId, infoId):
        regAddr = self.getAddress(self.BANK_RMM, triggerId)
        print("[set RM MAP] @", hex(regAddr), " info ", hex(infoId))
        self.write(regAddr, infoId)
        
    ####### Reconfig Module Map bank2
    # | BS_ID0 | CT_ID0 |
    # | BS_ID1 | CT_ID1 |
    # | BS_ID2 | CT_ID2 |
    # .
    def getRMInfo(self, infoId):
        bsIdxAddr = self.getAddress(self.BANK_RMINFO, infoId * 2)
        ### *2 because each row has two element
        ctrlAddr  = self.getAddress(self.BANK_RMINFO, (infoId * 2) + 1)
        print("[get RM INFO] bsIdxAddr@", hex(bsIdxAddr), " ctrlAddr@", hex(ctrlAddr))
        return (self.read(bsIdxAddr), self.read(ctrlAddr))
    
    def setRMInfo(self, infoId, bsIdx, ctrlCmd):
        ### *2 because each row has two element
        bsIdxAddr = self.getAddress(self.BANK_RMINFO, infoId * 2)
        ctrlAddr  = self.getAddress(self.BANK_RMINFO, (infoId * 2) + 1)
        print("[get RM INFO] bsIdxAddr@", hex(bsIdxAddr), " ctrlAddr@", hex(ctrlAddr))
        self.write(bsIdxAddr, bsIdx)
        self.write(ctrlAddr, ctrlCmd)
    ####### BIN stream bank3
    # | BIN_ADDR0 | SIZE0 |
    # | BIN_ADDR1 | SIZE1 |
    # | BIN_ADDR2 | SIZE2 |
    # .
    def getBSInfo(self, bsId):
        ### *3 because each row has three element
        streamIdentAddr  = self.getAddress(self.BANK_BSINFO, (bsId * 4)    )
        streamAddr       = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 1)
        sizeAddr         = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 2)
        print("[get BS INFO] streamAddr@", hex(streamAddr), " sizeAddr@", hex(sizeAddr))
        return (self.read(streamIdentAddr), self.read(streamAddr), self.read(sizeAddr))
    
    def setBSInfo(self, bsId, phyStreamAddr, streamSize):
    
        streamIdentAddr  = self.getAddress(self.BANK_BSINFO, (bsId * 4)    )
        streamAddr       = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 1)
        sizeAddr         = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 2)
        print("[get BS INFO] streamAddr@", hex(streamAddr), " sizeAddr@", hex(sizeAddr))
        self.write(streamIdentAddr, 1)
        self.write(streamAddr     , phyStreamAddr)
        self.write(sizeAddr       , streamSize)
        
    ######## AUTO META DATA RECONFIGURE
    def setSimpleMetaData(self, idx, streamPhyAddr, streamPhySize):
        
        print("setting RM Mapping to ", idx)
        self.setRMM(idx, idx)
        print("setting RM INFO to ", idx)
        ctrlValue = 0B0_10_0_00
        print("control value for active low reset is ", hex(ctrlValue))
        self.setRMInfo(idx, idx, ctrlValue)
        
        print("setting BS INFO to ", idx, " with streamAddress: ", streamPhyAddr, " with size: ", streamPhySize)
        self.setBSInfo(idx, streamPhyAddr, streamPhySize)
    
    ###########################################
    ######## DEBUGGER #########################
    ###########################################

    def printStatus(self):
        
        status = self.getStatus()
        
        print(">>status of the system vs0")
        print("-------> Is device shutdown: ", (status >> 7) & 0x1)
        print("-------> current error code: ", hex((status >> 3) & 0xF))
        print("-------> active RM_ID      : ", hex((status >> 8) & 0xFFFF))
        print("-------> state      : ", hex(status & 0x7))
        
    def printSimpleMetaData(self, idx):
        print("get metadata info for row", idx)
        print("RM MAPPER: ", self.getRMM   (idx))
        print("RM INFO  : ", self.getRMInfo(idx))
        print("BS INFO  : ", self.getBSInfo(idx))
        
        
        
    ###############################################
    ####### ALLOCATE BIN STREAM ON MAIN MEMORY ####
    ###############################################
    
    def allocateBitStreamCMA(self, path):
              
        print(">>allocateBitStream")
        
        print("opening file ", path)
        with open(path, 'rb') as f:
            data = f.read()
        file_size = len(data)
        print("copying the data")
        data_u32  = np.frombuffer(data, dtype='<u4')  # Big-endian uint32
        buffer    = allocate(shape=(len(data_u32),), dtype='>u4')
        buffer[:] = data_u32
        print("copy complete")
        print("file size ", file_size)
        print("---------------------------------")
        return buffer, buffer.physical_address, file_size

## memory allocation

In [5]:
def allocDataUint(allocShape = (16, ), allocType = np.float32, inputX = None):
    buf0 = allocate(shape=allocShape, dtype=allocType)
    #### copy the data
    if inputX is not None:
        print("start copy from input to allocate buffer")
        if (allocShape != inputX.shape) or (allocType != inputX.dtype):
            raise Exception("the specified shape and inputX shape is mismatch")
        np.copyto(buf0, inputX)
        print("copy finish")

    return buf0, buf0.physical_address, buf0.nbytes

## main procedure

In [6]:
AMT_SLOT = 3

In [7]:
#### change configmode
changePLconfigMode("pcap", True)

CHANGE CMD STDOUT: 0xFFCA3008 0xFFFFFFFF 0x1

CHANGE CMD ERROR : 
--------------------------------
TRIGGER CMD STDOUT: 0xFFCA3008

TRIGGER CMD ERROR : 
--------------------------------
READ CMD STDOUT: 0x1

READ CMD ERROR : 
--------------------------------


In [8]:
#### load overlay
overlay  = Overlay(PRJ_HW_DIR + FULL_BS_NAME)

In [9]:
#### get the device
dmaIp      = overlay.axi_dma_0
dfxCtrlIp  = overlay.dfx_controller_0
magicSeqIp = overlay.MagicSeqTop_0

{'type': 'xilinx.com:ip:dfx_controller:1.0', 'mem_id': 's_axi_reg', 'memtype': 'REGISTER', 'gpio': {}, 'interrupts': {}, 'parameters': {'C_XDEVICEFAMILY': 'zynquplus', 'ALL_PARAMS': 'HAS_AXI_LITE_IF 1 RESET_ACTIVE_LEVEL 0 CP_FIFO_DEPTH 32 CP_FIFO_TYPE lutram CDC_STAGES 6 VS {par0 {ID 0 NAME par0 RM {subRm {ID 0 NAME subRm BS {0 {ID 0 ADDR 0 SIZE 0 CLEAR 0}} RESET_REQUIRED low SHUTDOWN_REQUIRED hw} ml1 {ID 1 NAME ml1 BS {0 {ID 0 ADDR 0 SIZE 0 CLEAR 0}} SHUTDOWN_REQUIRED hw RESET_REQUIRED low} ml2 {ID 2 NAME ml2 BS {0 {ID 0 ADDR 0 SIZE 0 CLEAR 0}} SHUTDOWN_REQUIRED hw RESET_REQUIRED low} ml3 {ID 3 NAME ml3 BS {0 {ID 0 ADDR 0 SIZE 0 CLEAR 0}} SHUTDOWN_REQUIRED hw RESET_REQUIRED low}} START_IN_SHUTDOWN 1 NUM_HW_TRIGGERS 4 NUM_TRIGGERS_ALLOCATED 4}} CP_FAMILY ultrascale_plus DIRTY 2', 'Component_Name': 'system_dfx_controller_0_0', 'GUI_BS_ADDRESS_0': '0x00000000', 'GUI_BS_ADDRESS_1': '0x00000000', 'GUI_BS_CLEAR_0': 'false', 'GUI_BS_CLEAR_1': 'true', 'GUI_BS_SIZE_0': '0', 'GUI_BS_SIZE_1': '0